In [1]:
from AOC_utils import get_day
import numpy as np

day = 20
input_data = get_day(day)

# print out first few lines to get a feel for the data
input_data[:7]

Day 20 input already downloaded


['%fl -> tf, gz',
 '%xb -> hl, tl',
 '%mq -> tf, fl',
 '%px -> hl, tm',
 '%dp -> xv',
 'broadcaster -> js, ng, lb, gr',
 '&ql -> rx']

In [2]:
example = '''
broadcaster -> a, b, c
%a -> b
%b -> c
%c -> inv
&inv -> a
'''.split('\n')[1:-1]

In [3]:
example = '''
broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output
'''.split('\n')[1:-1]

In [4]:
len(input_data)

58

In [5]:
import math

In [48]:
def parse_data(data):

    def initialize_modules():
        modules = {}
        conjuncts = []
        for line in data:
            operator = line[0]
            name, outputs = line.split(' -> ')
            if not 'broadcaster' in name:
                name = name[1:]
            outputs = outputs.split(', ')
            if operator == '%':
                state = False # flip flops start off
            elif operator == '&':
                conjuncts.append(name)
                state = {}
            else:
                state = False
            modules[name] = (operator, outputs, state)

        # need to fill in conjunct inputs
        for conjunct in conjuncts:
            for key, values in modules.items():
                if conjunct in values[1]:
                    modules[conjunct][2][key] = False

        # need to add empty modules that don't yet exist
        add_modules = []
        for module in modules:
            operator, outputs, state = modules[module]
            for output in outputs:
                if output not in modules:
                    add_modules.append(output)
        for module in add_modules:
            modules[module] = ('b', [], False)

        return modules

    modules = initialize_modules()
    
    stack = []
    signal_cache = {}
    def pass_signal(name, signal, signal_from, order=0):
        operator, outputs, state = modules[name]
        # state_string = ""
        # if type(state) == dict: state_string.join([str(int(x)) for x in state.values()])
        # else: state_string = str(int(state))
        # signal_string = name + signal_from + operator + str(int(signal)) + str(state) + state_string
        # if signal_string in signal_cache:
        #     state, output_stack = signal_cache[signal_string]
        #     modules[name] = (operator, outputs, state)
        #     stack.insert(order, output_stack)
        # else:
        output_stack = []
        if operator == '%' and not signal:
            state = not state
            modules[name] = (operator, outputs, state)
            for output in outputs:
                output_stack.append((output, state, name))
        elif operator == '&':
            state[signal_from] = signal
            modules[name] = (operator, outputs, state)
            for output in outputs:
                output_stack.append((output, not all(state.values()), name))
        elif operator == 'b': # broadcaster
            for output in outputs:
                output_stack.append((output, signal, name))
        stack.insert(order, output_stack)
            # signal_cache[signal_string] = (state, output_stack)
        if name == "ql" and signal:
            # print("ql", state)
            return state.copy()
        else:
            return False


    def push_button():
        state_outputs = []
        low_pulses = 0
        high_pulses = 0
        stack.append([('broadcaster', False, "button")])
        while stack:
            substack = stack.pop(0)
            for i in range(len(substack)):
                name, signal, signal_from = substack[i]
                # print(signal_from, signal, name)
                state_output = pass_signal(name, signal, signal_from, order=i)
                if state_output:
                    state_outputs.append(state_output)
                if signal:
                    high_pulses += 1
                else:
                    low_pulses += 1
        return low_pulses, high_pulses, state_outputs
        
    global_low_pulses = 0
    global_high_pulses = 0
    for i in range(1000):
        low_pulses, high_pulses, _ = push_button()
        global_low_pulses += low_pulses
        global_high_pulses += high_pulses

    total = global_low_pulses * global_high_pulses
    print("part 1:", total)


    # part 2
    modules = initialize_modules()
    # print(len(modules))
    

    previous_rx_states = np.full(4, False)
    
    cycle_has_started = np.full(len(previous_rx_states), 0)
    cycle_starts = np.full(len(previous_rx_states), None)
    cycle_ends = np.full(len(previous_rx_states), None)

    # print(previous_rx_states)
    
    for i in range(100000):
        low_pulses, high_pulses, ql_state_change = push_button()
        # check conjunctor states
        if ql_state_change:
            new_states = list(ql_state_change[0].values())
            # print("new_states", new_states)
        else:
            new_states = np.full(4, False)
        for idx in range(4):
            if previous_rx_states[idx] != new_states[idx]:
                if new_states[idx]:
                    if cycle_has_started[idx] == 0:
                        cycle_starts[idx] = i
                    elif cycle_has_started[idx] == 1:
                        cycle_ends[idx] = i
                    cycle_has_started[idx] += 1
        
        previous_rx_states = new_states


        # if i % 10000 == 0:
            # print(i, cycle_starts, cycle_ends)
        if all(cycle_ends):
            break

            # print(previous_rx_states)

    # print(modules_to_bytes())
    
    # print(cycle_starts)
    # print(cycle_ends)
    cycle_lengths = np.array(cycle_ends) - np.array(cycle_starts)
    # print(cycle_lengths)
    # print(cycle_lengths * 2)

    # print(math.lcm(*cycle_starts[cycle_starts != 0]))
    # print(math.lcm(math.lcm(*cycle_starts[cycle_starts != 0]), math.lcm(*cycle_lengths)))
    # print(cycle_lengths)
    total2 = math.lcm(*cycle_lengths)
    print("part 2:", total2)

In [49]:
parse_data(input_data)

part 1: 787056720


TypeError: unsupported operand type(s) for -: 'NoneType' and 'NoneType'

In [47]:
parse_data(example)

part 1: 11687500


TypeError: unsupported operand type(s) for -: 'NoneType' and 'NoneType'